**Task 07: Querying RDF(s)**

In [2]:
!pip install rdflib 
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2020-2021/master/Assignment4"

     |████████████████████████████████| 235kB 2.8MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 


Leemos el fichero RDF de la forma que lo hemos venido haciendo

In [3]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/resources/example6.rdf", format="xml")

<Graph identifier=N26ec8930471b4301a308de6db7921feb (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "Person" with RDFLib and SPARQL**

In [4]:
from rdflib.plugins.sparql import prepareQuery
ns = Namespace("http://somewhere#")

print("RDFLib list")
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  print(s,p,o)

print("SPARQL list")
q1 = prepareQuery('''
  SELECT ?sco
  WHERE { 
    ?sco rdfs:subClassOf ns:Person. 
  }
  ''',
  initNs = { "ns": ns}
)
for s in g.query(q1):
  print(s)

RDFLib list
http://somewhere#Researcher http://www.w3.org/2000/01/rdf-schema#subClassOf http://somewhere#Person
SPARQL list
(rdflib.term.URIRef('http://somewhere#Researcher'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [15]:
print("RDFLib list")
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  print(s)
for s1,p1,o1 in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s2,p2,o2 in g.triples((None, RDF.type, s1)):
    print(s2)

print("SPARQL list")
q2 = prepareQuery('''
  SELECT ?p
  WHERE { 
    { 
      ?p rdf:type ns:Person.
    } UNION {
      ?sco rdfs:subClassOf ns:Person.
      ?p rdf:type ?sco
    }
  }
  ''',
  initNs = { "ns": ns }
)
for s in g.query(q2):
  print(s)

RDFLib list
http://somewhere#JohnSmith
http://somewhere#SaraJones
http://somewhere#JaneSmith
SPARQL list
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)


**TASK 7.3: List all individuals of "Person" and all their properties including their class with RDFLib and SPARQL**


In [22]:
print("RDFLib list")
for s1,p1,o1 in g.triples((None, RDF.type, ns.Person)):
  for s2,p2,o2 in g.triples((s1, None, None)):
      print(s2,p2,o2)
for s3,p3,o3 in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s4,p4,o4 in g.triples((None, RDF.type, s3)):
    for s5,p5,o5 in g.triples((s4, None, None)):
      print(s5,p5,o5)

print("SPARQL list")
q3 = prepareQuery('''
  SELECT ?s ?p ?o
  WHERE { 
    { 
      ?s rdf:type ns:Person.
      ?s ?p ?o
    } UNION {
      ?s rdf:type ?sco.
      ?s ?o ?p.
      ?sco rdfs:subClassOf ns:Person
    }
  }
  ''',
  initNs = { "ns": ns}
)

for s in g.query(q3):
  print(s)

RDFLib list
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#JaneSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Researcher
http://somewhere#JaneSmith http://somewhere#http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#JaneSmith http